Aqui se explica el proceso para poder utilizar la red generativa entrenada en este proyecto

In [5]:

import os
import random
from PIL import Image,  ImageOps
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.layers import Input, Dense
import time
import torch
import torchvision
import torch.nn as nn
import torchvision.datasets as datasets
from torch.utils.data import Subset
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import copy
from sklearn.metrics import confusion_matrix
from torch.utils.data import RandomSampler, ConcatDataset
import pandas as pd
import shutil
from PIL import ImageFile
from tqdm import tqdm
from PIL import ImageEnhance


# Construir el modelo

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Usando GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("GPU no disponible, usando CPU")

Usando GPU: NVIDIA GeForce GTX 1060 6GB


In [7]:
def initialize_model2(num_classes):
    #Los pesos si se entrenan
    model_ft = models.inception_v3(weights=False) #Se carga el modelo inception_v3 sin sus pesos preentrenados.
    # False en las redes convencionales, se entrenaran
    set_parameter_requires_grad(model_ft, False)
    # Se modifica la red auxiliar
    num_ftrs = model_ft.AuxLogits.fc.in_features
    model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)

    #Se modifica la red principal para que sea de dos clases con una salida softmax y coger la con mas probabilidad.
    model_ft.dropout= nn.Dropout(0.5)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Sequential(
        nn.Linear(num_ftrs,num_classes),
        nn.Dropout(0.3),
        nn.Softmax(dim=1)
    )

    input_size=299 #Tamaño que usa la red
    return model_ft,input_size

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
device = torch.device("cpu")
Modeloft,_= initialize_model2(2)
Modelo=Modeloft.to(device)

#Para inutilizar la capa auxiliar.
Modeloft.aux_logits=False

In [ ]:
PATH = '..\Pesos\modeltranformaciones.pt'
Modelo.load_state_dict(torch.load(PATH))

# Cargar el peso

In [16]:
PATH = '..\Pesos\Discriminador\mmodeltranformacionesEpocaBestEpoch.pt'
Modelo.load_state_dict(torch.load(PATH))

<All keys matched successfully>

# Uso de la red

In [17]:
transform = transforms.Compose([
    transforms.Resize((299,299)), #El primer componente es la altura de la imagen y el segundo es su ancho
    transforms.ToTensor() #Transforma las imagenes a tensor
])

In [ ]:
ruta_imagen_prueba = '../ImagenPrueba/Discriminador/Input/im_f1000_.JPG' # Se pone la ruta de la imagen que se desee modificar

# Cargar la imagen de prueba 
imagen_prueba = Image.open(ruta_imagen_prueba)
imagen_prueba = transform(imagen_prueba).unsqueeze(0) 
imagen_prueba = imagen_prueba.to(device)
with torch.no_grad():
    Modelo.eval()  # Poner el generador en modo de evaluación
    puntuacion = Modelo(imagen_prueba)[:, 1]

print(f'Puntuacion imagen :{puntuacion.item()}')


# Si es mayor a 0.5 es de mala calidad y si es inferior es de buena calidad.